## Kyoto2006+ データセットを題材にした K-means を用いたクラスタリングによる予測モデルの生成と評価

### 準備とデータの読み込み
必要なパッケージを読み込む

In [8]:
# from sklearn import patch_sklearn
# patch_sklearn()
import pandas
import glob
from time import time
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

Kyoto2006+ データセットは CSV 形式でデータが格納されており、それを表形式に変換する。変換後のファイルの各列の名前とデータのタイプの順をここで指定する。

In [9]:
column = [
    ["duration", "float32"],
    ["service", "object"],
    ["source_bytes", "uint32"],
    ["destination_bytes", "uint32"],
    ["count", "uint32"],
    ["same_srv_rate", "float32"],
    ["serror_rate", "float32"],
    ["srv_serror_rate", "float32"],
    ["dst_host_count", "uint32"],
    ["dst_host_srv_count", "uint32"],
    ["dst_host_same_src_port_rate", "float32"],
    ["dst_host_serror_rate", "float32"],
    ["dst_host_srv_serror_rate", "float32"],
    ["flag", "object"],
    ["ids_detection", "object"],
    ["malware_detection", "object"],
    ["ashula_detection", "object"],
    ["label", "int8"],
    ["source_ip_address", "object"],
    ["source_port_number", "uint16"],
    ["destination_ip_address", "object"],
    ["destination_port_number", "uint16"],
    ["start_time", "object"],
    ["protocol", "object"]
]
column_names = [i[0] for i in column]
column_types = { i[0]: i[1] for i in column}

データセットを読み込む。時間の短縮のため、2015年2月分のみを用いて演習の手順を説明する。
成果発表では、できるだけデータセット全体を用いて評価してみること。

今回はデータ分析のためのツールである Pandas (https://pandas.pydata.org/) を利用する。

`read_csv` はデータ間がカンマで区切られた txt ファイルからデータを読み込む API である。
第1引数はファイルのパス、`sep`は区切り文字の指定、`header`はtxtファイルの中で列名が入っている行番号、`names`は列名のリストを指定する。CSVファイルには列名は入っていないので `None` を、`names` は `column` のリストを利用する。

In [11]:
# 2015/02の各日データがあるtxtファイルのパスを配列として取得
files = glob.glob("./dataset/kyoto2006plus/Kyoto2016/2015/02/201502*.txt")
# 時系列順にソートする
files.sort()
# 各日データを結合して一つのデータセットとして読み込む
kyoto_data = pandas.concat([pandas.read_csv(x, sep='\t', header=None, names=column_names, dtype=column_types) for x in files], ignore_index=True)

データの数や平均、分散などの性質を確認するには、`describe` メソッドを利用する。

In [12]:
kyoto_data.describe()

,duration,source_bytes,destination_bytes,count,same_srv_rate,serror_rate,srv_serror_rate,dst_host_count,dst_host_srv_count,dst_host_same_src_port_rate,dst_host_serror_rate,dst_host_srv_serror_rate,label,source_port_number,destination_port_number
count,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06
mean,1.317268e+00,5.492566e+04,2.175131e+03,5.175024e+00,4.744613e-01,4.152513e-02,4.192815e-01,3.506128e+01,4.014791e+01,2.828774e-02,8.570371e-02,1.340752e-01,-9.081160e-01,3.344987e+04,2.309937e+03
std,7.953889e+01,8.052808e+06,1.331901e+06,1.023963e+01,4.968409e-01,1.955620e-01,4.526754e-01,4.195258e+01,4.327172e+01,1.617517e-01,2.684495e-01,3.284109e-01,4.189721e-01,2.062878e+04,7.910031e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,1.220000e+04,2.300000e+01
50%,4.210000e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,1.100000e+01,1.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,3.795400e+04,5.300000e+01
75%,1.435795e+00,6.500000e+01,1.180000e+02,4.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,9.200000e+01,9.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,5.122700e+04,4.450000e+02
max,8.392452e+04,2.121764e+09,1.583454e+09,1.000000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+02,1.000000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.553500e+04,6.553500e+04


それぞれのフローが攻撃かどうかは "label" 列に入っている。

1は正常な通信、その他は不正な通信であり、不正な通信はいくつかの種類(-1:known attack, -2:unknown attack)に分かれている。

label の種類数やそれぞれの label の数を表示するには、値とその値を持つデータの数を出力する `value_counts()` メソッドを利用する。

In [13]:
kyoto_data['label'].value_counts()

label
-1    7513100
 1     362107
-2        557
Name: count, dtype: int64

### K-means の適用
データセットの中には、クラスタリングにそのまま用いることは困難な（そのままでは距離を定義することができない）データがある。
例えば、protocol はプロトコルの種別が入っており、量ではないので、異なる種別間の距離は何らかの方法で定義する必要がある。
そのため、クラスタリングを実行する前に、データに前処理を施す必要がある。

今回は、そのようなデータを除いてクラスタリングを行う。クラスタリングに利用する列は以下とする。

In [14]:
cluster_features = [
    "duration", "source_bytes", "destination_bytes", 
 "count", "same_srv_rate", "serror_rate", "srv_serror_rate","dst_host_count",
  "dst_host_srv_count", "dst_host_same_src_port_rate", "dst_host_serror_rate",
  "dst_host_srv_serror_rate", "source_port_number", "destination_port_number"
]

この列のみを抽出する。
また、protocol 種別のような種類を示す column はダミー変数にするとよい。
cluster_features を抽出し、protocol をダミー変数として追加した `cluster_data` を取得する。

In [15]:
cluster_data = pandas.get_dummies(kyoto_data[cluster_features + ['protocol']])

のようにする。

それぞれの列の値が取り得る値域は大きく異なるため、値域の幅が大きいものに結果が大きく影響してしまう可能性がある。そこで、`sklearn.preprocessing.MinMaxScaler` を利用し、最小値を0、最大値を1に正規化する。

具体的には、MinMaxScaler の `fit_transform` メソッドを利用する。

この返り値は numpy の array なので、Pandas で扱うには Pandas の DataFrame に戻す必要がある。

In [16]:
cluster_data = pandas.DataFrame(MinMaxScaler().fit_transform(cluster_data))

`describe` メソッドで最小値が0, 最大値が1になっていることが確認できる。

In [17]:
cluster_data.describe()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06,7.875764e+06
mean,1.569586e-05,2.588679e-05,1.373662e-06,5.175024e-02,4.744610e-01,4.152513e-02,4.192817e-01,3.506128e-01,4.014791e-01,2.828774e-02,8.570375e-02,1.340752e-01,5.104123e-01,3.524738e-02,1.113289e-02,6.184246e-01,3.704425e-01
std,9.477432e-04,3.795336e-03,8.411361e-04,1.023963e-01,4.968409e-01,1.955620e-01,4.526754e-01,4.195258e-01,4.327172e-01,1.617517e-01,2.684495e-01,3.284109e-01,3.147750e-01,1.206993e-01,1.049235e-01,4.857732e-01,4.829233e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.861601e-01,3.509575e-04,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.016413e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-02,1.100000e-01,1.600000e-01,0.000000e+00,0.000000e+00,0.000000e+00,5.791409e-01,8.087282e-04,0.000000e+00,1.000000e+00,0.000000e+00
75%,1.710817e-05,3.063489e-08,7.452062e-08,4.000000e-02,1.000000e+00,0.000000e+00,1.000000e+00,9.200000e-01,9.600000e-01,0.000000e+00,0.000000e+00,0.000000e+00,7.816739e-01,6.790265e-03,0.000000e+00,1.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


このデータに対して、K-means を実行する。`n_clusters` はクラスタ数を指定する。ここでは 30 を指定する。データ量の関係から Mini Batch K-means を利用する。

In [18]:
n_clusters = 30
km = MiniBatchKMeans(n_clusters=n_clusters, batch_size=10000).fit(cluster_data)

/Users/ansai/.pyenv/versions/3.9.6/envs/security-practice/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


### 結果の確認
元のデータのラベルを参考に、各クラスタに属するデータのラベルの種類や数を見ていく。

元々のデータセットの各行がどのクラスタに分類されたかは、`km.labels_` の配列に入っている。

データセットの各行のラベルをクラスタ毎に類別する。類別結果を`label_names`に格納する。

In [19]:
label_names = [ kyoto_data['label'][km.labels_ == cluster_index] for cluster_index in range(n_clusters)]

In [13]:
for idx, val in enumerate(label_names):
    print("Cluster {}".format(idx))
    print(val.value_counts())
    print()

Cluster 0
-1    569734
 1       899
Name: label, dtype: int64

Cluster 1
-1    969836
 1     67156
Name: label, dtype: int64

Cluster 2
-1    120387
-2         3
 1         2
Name: label, dtype: int64

Cluster 3
-1    782465
 1     58060
-2       250
Name: label, dtype: int64

Cluster 4
-1    441941
 1       768
Name: label, dtype: int64

Cluster 5
-1    165975
 1     24738
Name: label, dtype: int64

Cluster 6
-1    342069
 1      9925
Name: label, dtype: int64

Cluster 7
-1    23723
 1      145
-2        3
Name: label, dtype: int64

Cluster 8
-1    72887
 1     6533
Name: label, dtype: int64

Cluster 9
-1    142290
-2        48
 1        20
Name: label, dtype: int64

Cluster 10
-1    155723
 1       529
-2         2
Name: label, dtype: int64

Cluster 11
-1    73755
 1    23452
Name: label, dtype: int64

Cluster 12
-1    77156
 1    10512
Name: label, dtype: int64

Cluster 13
-1    146233
 1     45158
Name: label, dtype: int64

Cluster 14
-1    67190
 1      234
Name: label, dtype: int

例えば、ここから各クラスタに含まれる正常な通信と不正な通信の割合を見ていくなら、以下のようにすればよい。normal と attack が綺麗に分かれているほどよい。

In [20]:
for idx, val in enumerate(label_names):
    print("Cluster {}".format(idx))
    attack = 0
    normal = 0
    for label in val:
        if label == 1:
            normal += 1
        else:
            attack += 1
    print("normal = {}, attack = {}, total = {}".format(normal/(normal+attack), attack/(normal+attack), normal+attack))

Cluster 0
normal = 0.14022069807761506, attack = 0.8597793019223849, total = 183418
Cluster 1
normal = 0.0019365397724432835, attack = 0.9980634602275568, total = 940337
Cluster 2
normal = 0.06979320654984966, attack = 0.9302067934501503, total = 899013
Cluster 3
normal = 0.0680232353458898, attack = 0.9319767646541102, total = 852150
Cluster 4
normal = 0.0, attack = 1.0, total = 232933
Cluster 5
normal = 0.00409011469944633, attack = 0.9959098853005537, total = 136182
Cluster 6
normal = 0.0022064690220265897, attack = 0.9977935309779734, total = 892376
Cluster 7
normal = 0.025191064008567826, attack = 0.9748089359914321, total = 411773
Cluster 8
normal = 0.24241293918953924, attack = 0.7575870608104608, total = 96513
Cluster 9
normal = 0.00029693647512961667, attack = 0.9997030635248704, total = 255947
Cluster 10
normal = 0.022631243850205474, attack = 0.9773687561497946, total = 310986
Cluster 11
normal = 0.1199069215677328, attack = 0.8800930784322671, total = 87668
Cluster 12
norma

クラスタリングされた結果の評価尺度は様々なものがあるが、クラスタの中心とそのクラスタに分類されたデータとの距離の和は `inertia_` 属性でアクセスできる。

In [21]:
km.inertia_

830938.5553672388

### 検証
本演習の目的は、あるフローが与えられた時に正規の通信か不正な通信かを予測する予測モデルを過去のトラフィックデータから生成することである。

ここでは、過去の通信から現在の通信が正規なものか不正なものかを判断するので、時間的な順序を考慮し、データセットを分割し、過去のトラフィックデータで学習させたもので新しい通信の予測がどの程度正しいかどうかの検証を行う。

まずはデータセットのラベルを attack と normal の2種類に変換する関数を用意する。`get_label`関数は、あるクラスタに属するデータのラベルが配列として入力され、その配列のうち正常(1)と攻撃(-1,-2)どちらが多いか判断し、その結果を出力する。

In [22]:
def get_label(cluster):
    normal = 0
    attack = 0
    ret = "unknown."
    for label in cluster:
        if label == 1:
            normal += 1
        else:
            attack += 1
    if normal > attack:
        ret = 'normal.'
    elif normal < attack:
        ret = 'attack.'
    return ret

データセットを4分割し、過去の3セットで学習し、それより新しい1セットの予測を検証する。

In [23]:
data_len = int(len(cluster_data)/4)
learn_data = 3

過去の3セットでクラスタリングを行い、各クラスタに normal または attack のラベルをつける。

In [24]:
km.fit(cluster_data[0:data_len * learn_data])
label_names = [ get_label(kyoto_data['label'][:data_len * learn_data][(km.labels_ == cluster_index)]) for cluster_index in range(n_clusters)]

/Users/ansai/.pyenv/versions/3.9.6/envs/security-practice/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


学習に用いなかったデータの予測を行う。
predにはどのクラスタに属するかの値が入る

In [25]:
pred = km.predict(cluster_data[data_len * learn_data:])

予測が正しいかどうかを検証する。

In [26]:
success = 0
fail = 0
normal = 0
attack = 0
# correct_predict
tp = 0 # 真陽性(true positive) ：　正しく陽性と判定
fn = 0 # 偽陰性(false negative)　：　本当は陽性なのに、陰性と判定
fp = 0 #偽陽性(false positive)　：　本当は陰性なのに、陽性と判定
tn = 0 #真陰性(true negative)　：　正しく陰性と判定

for i in range(data_len * (4 - learn_data)):
    predicate = label_names[pred[i]]
    correct = get_label([kyoto_data['label'][data_len * learn_data + i]])
    if predicate == correct: # 正しく判定
        success += 1
        # 真陽性か真陰性か
        if correct == 'normal.':
            tp += 1 # 正しく陽性と判定
        elif correct == 'attack.':
            tn += 1 # 正しく陰性と判定
    else: # 誤った判定
        fail += 1
        # 偽陰性か偽陽性か
        if correct == 'normal.':
            fn += 1# 誤って陰性と判定
        elif correct == 'attack.':
            fp += 1# 誤って陽性と判定
    # 正常通信と攻撃通信の数を計算
    if correct == 'normal.':
        normal += 1
    elif correct == 'attack.':
        attack += 1

print("success = {}, failed = {}, normal = {}, attack = {}, unknown = {}".format(success, fail, normal, attack, success + fail - normal - attack))
print("TP = {}, FN = {}, FP = {}, TN = {}".format(tp, fn, fp, tn))

success = 1937256, failed = 31685, normal = 31685, attack = 1937256, unknown = 0
TP = 0, FN = 31685, FP = 0, TN = 1937256


また、予測が正しいかの検証は以下のプログラムを実行しても確認できる。混同行列と呼ばれる2×2行列の各成分を見ることで、TPの値などを調べられる。
混合行列である`confusion_matrix`から各成分の値を取り出す際は、`ravel()`メソッドを用いれば良い。


$$
    ConfusionMatrix = \left[\begin{array}{c|c} TN & FP \\ \hline FN & TP \\ \end{array}\right]
$$

In [27]:
from sklearn.metrics import confusion_matrix
correct = kyoto_data['label'][data_len * learn_data:].map(lambda x: 'normal.' if x == 1 else 'attack.')
predicate = pandas.Series(pred).map(lambda x: label_names[x])

c_matrix = confusion_matrix(correct, predicate)
tn, fp, fn, tp = c_matrix.ravel()

print(c_matrix)

[[1937256       0]
 [  31685       0]]


分類問題のモデルを評価する際に使われる代表的な評価指標
1. 正確度(Accuracy): 推定した値と真の値が一致した割合
$$ \frac{TP + TN}{TP + FP + TN + FN} = \frac{正しく判定}{全体}$$
    - FPとFNの重要度について考慮しなくていい場合に使える
2. 適合率(Precision): モデルが陽性と判定した中で、真に陽性だった割合 
$$ \frac{TP}{TP + FP} = \frac{真に陽性}{陽性と判定} $$
    - 明らかに陽性と分かりやすいものだけを見つけたい時
    - FNが発生することを許容できるようなケースで、それでもなおFPがあっては困る場合に使える
3. 再現率(Recall): 真に陽性だったものの中で、モデルが陽性と判定した割合 
$$ \frac{TP}{TP + FN} = \frac{陽性と判定}{真に陽性} $$
    - 怪しいものを全て見つけ出したい時
    - FPが発生することを許容できるケースで、それでもなおFNがあっては困る場合に使える。
4. F-値(F-measure): 適合率と再現率の調和平均
$$ \frac{2}{\frac{1}{Precision} + \frac{1}{Recall}} =  \frac{2 * (Precision * Recall)}{Precision + Recall} $$
    - 正確度と違い、陽性と陰性の出現度合いが極端に異なる場合でも、評価しやすい
5. 特異度(Specificity): 実際に陰性だったものの中で、モデルが陽性と判断した割合
$$ \frac{TN}{FP + TN} $$

In [28]:
print("Accuracy = {}" .format((tp + tn) / (tp + tn + fp + fn) ))
if tp > 0 or fp > 0:
    print("Precision = {}" .format( tp/ (tp + fp) ))
print("Recall = {}" .format(tp/ (tp + fn) ))
print("F-measure = {}" .format(2 * tp / (2 * tp + fn + fp ) ))
print("Specificity = {}" .format(tn/(fp + tn) ))

Accuracy = 0.9839075929649492
Recall = 0.0
F-measure = 0.0
Specificity = 1.0


実際には、学習および検証に使うデータの時期を少しずつ変更して何度も評価が行われる。
例えば、利用するデータセットを n 週間分 (n = 1,...) ずらして同様の評価を行うなどが考えられる。

### 改良に向けたアイデア
以下はあくまでも例であり、各グループでアイデアを出し合って様々な改良を試すことを期待している。
- 結果を見てわかるように、データセットのlabelには大きな偏りがあり（攻撃トラフィックが圧倒的に多い）、上の手順ではこれを考慮していないため、ほぼ全ての通信が攻撃と判定されている。これでは現実的には役に立たない。そこで、不均衡なデータを扱うための枠組み（サンプリングなど）を適用してみる。
  - 例えば、攻撃トラフィックのみを K-means でクラスタリングし、各クラスタから少数のトラフィックだけサンプリングしてみるなど。
- 通信のパターンはサービス (http, ftp など) によって異なると考えられるため、サービスごとに学習させてみる。
- ダミー変数にするフィールドを追加・削除してみる。あるいは、一部のフィールドを削除してみる。
- 教師あり学習 (SVM など) を利用してみる。SVMを利用した例は kyoto2006plus_svm_example.ipynb にある (TA提供)。
- 正常な通信と攻撃の通信ではなく、既知の攻撃か未知の攻撃かを予測する。

なお、攻撃者は検知システムを迂回するために攻撃手法に少しずつ変更を加えてくることが想定される。検知システムはそれらの変更に頑健であることが求められる。例えば、IPアドレスをベースに検知することは不適切である一方、攻撃のホストの振る舞いを学習するために、同一のホストが行なった複数の通信を紐づけるためのkeyとしてIPアドレスを用いることは問題ないであろう。これは、攻撃に利用するIPアドレスを変更することは容易であるが、攻撃による通信の振る舞いを変更することは (おそらく) 難しいためである。